# Example of how to use the general Machine Learning Functions in general_ML_utils.py

## Importing the module


the script in which you want to import general_ML_utils has to be located in the same root folder as general_ML_utils.

Then import it by:

In [ ]:
import general_ML_utils as gmlu

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image 
from tqdm.auto import tqdm
from datetime import datetime

## Building a dataset


In order to use a dataset to train a model, the data must be in a specific form and format. A function that uses a folder of images and labels and constructs a usable dataset is often called a 'pipeline'. 

Such a pipeline is the function build_img_dataset_pipeline().

The function builds a pipeline that formats the images and labels and splits the dataset into training and validation and test sets.

### Downloading Dataset
 

To try the function we first have to have a dataset. For that we download the popular Fashion MNIST Dataset https://github.com/zalandoresearch/fashion-mnist?tab=readme-ov-file

We will download the images and save them to a folder in the root directory of this script. We only use the 10.000 images of the test set for faster execution. Normally this is not enough. A rough estimate is at least 1000 images per class.

In [ ]:
#we download the fashion mnist dataset from keras
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
#concatenate the training and testing data because we will split it later
# images = np.concatenate((train_images, test_images))
# labels = np.concatenate((train_labels, test_labels))
images = test_images
labels = test_labels
#create a folder to store the images
os.makedirs('fashion_mnist', exist_ok=True)
os.makedirs('fashion_mnist/images', exist_ok=True)
os.makedirs('fashion_mnist/labels', exist_ok=True)
print('Saving images...')
for i, image in tqdm(enumerate(images)):
    Image.fromarray(image).save('fashion_mnist/images/' + str(i) + '.png')
print('Done!')
print('Images saved in fashion_mnist/images')
#one hot encode the labels
labels = gmlu.one_hot_encode(labels, 10)
#make dataframe with first column the image names and the other columns the labels, each column corresponding to a class
image_names = [f'{i}.png' for i in range(len(images))]
data = np.column_stack([image_names, labels])
columns = ['image_name'] + [f'class_{i}' for i in range(10)]
df = pd.DataFrame(data, columns=columns)
# Save the DataFrame as a CSV file, this will be the labels file
df.to_csv('fashion_mnist/labels.csv', index=False)
print('Labels saved as fashion_mnist/labels.csv')
print('The first 5 rows of the labels file are:')
print(df.head())
train_images, test_images, train_labels, test_labels, images, labels = None, None, None, None, None, None

### Dataset pipeline

Now we have the raw dataset in the right form.

The images are saved in the folder fashion_mnist/images and the labels a saved in the file fashion_mnist/labels.csv.

The images are of size 28x28 and are greyscale. There are 10 classes.

We can now use the build_img_dataset_pipeline() of the gmlu function.

To see the documentation of a function with all possible parameters we can call help(function).


In [ ]:
help(gmlu.build_img_dataset_pipeline)

We now use the function. Since the function is saved in the module general_ML_utils.py which we imported as gmlu we call the function with gmlu.build_img_dataset_pipeline().

We use the function with augmentation_strength=0.3 for every augmentation except vertical and horizontal translation since all objects in the dataset are perfectly centered. This means that we randomly adjust brightness and contrast by up to 30% of the original value and add gaussian noise with up to 0.3 as stadard deviation (the exact value of the augmentation is random and changes each epoch. The augmentation_strength only represents the maximum possible value). We also flip images randomly vertically and horizontally.

augmentation_strength=0 would mean that we only use the original images, but augmentation always leads to betters results. Normal values are 0.1 or 0.2.

In [ ]:
img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
train_ds, val_ds, test_ds = gmlu.build_img_dataset_pipeline(img_path=img_path, label_path=label_path, img_size=(32, 32, 1), n_classes=10, img_type='png',augment_strength=[1, 0, 0, 0.3, 0.3, 0.3],seed=42)

In the above function the augmentations get applied dynamically. When you want to use the augmented images in a different software or programming language we can save the augmented dataset to a folder with the save_augmented_dataset() function.

With this function we copy the original images plus the augmented images to a new folder and generate the corresponding labels.

augment_iterations=2 means that we create 2 randomly augmented versions from each original image.

The original images keep their name and to the augmented images a '_aug' plus the iteration number is added.

In [ ]:
img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
save_path = r'fashion_mnist/augmented'
gmlu.save_augmented_dataset(img_path=img_path, label_path=label_path, save_path=save_path, augment_strength=0.1, augment_iterations=2, img_size=(32, 32, 1),img_type='png', n_classes=10)


### Übungen 1

1. Build a dataset with a train and val set of equal sizes and no test set and apply only the augmentations of brightness and gaussian noise with strength 0.9.

2. Build a dataset once with a halfed image-resolution ond once with a four time higher image-resolution and compare the results.

3. Build a dataset consisting of only a test set and name the first 20 image-filenames. Compare them when shuffling and not shuffling the data.

## Building a model


We use the build_model() function to compile a model for use. 

We can choose from several different model architectures which are: 'resnet', 'inceptionv3', 'mobilenetv2', 'xception','efficientnet'. The models can be constructed in the sizes 's', 'm', 'l'. Bigger models usually yield a better performance but this difference is small for small and easy datasets and increases the needed training time drastically.

We use the mobilenet architecture here because it is fast to train and leads to good results most of the time.

Each model can be loaded with pretrained weights, which were trained on the imagenet dataset. Using pretrained weights often leads to a better performance, but might have slightly negative effects when the used dataset is very different from the imagenet dataset. The weights cant be used if the images are greyscale. So we wont use them here.

In [ ]:
help(gmlu.build_model)

In [ ]:
model = gmlu.build_model(input_shape=(32, 32, 1), n_classes=10, model_type='simplenet',model_size='s',weights=None, classification_type='multiclass')

### Übungen 2

1. Build a EfficientNet in size s and count the number of Conv2D Layers. Compare with simpleNet in size L.

2. Build a s-sized simplenet with 4 classification layers with 128, 64, 32 and 16 neurons. Compare the total number of parameters with the same net but with 2 classification layers with 256 and 32 neurons.

3. Try to build a xception model with input_shape=(32,32,1). What is the result?

4. Try to build a resnet model with input_shape=(32,32,1) and use the imagenet weights. What is the result?

## Training a model


Here we train the compiled model with the dataset.

We wont get a very good result because the dataset is so small, but that enables a much faster training.

During the training we see the metrics accuracy, precision, recall and f1-score for each epoch. After each epoch we see all metrics for the validation set.

The model the function returns is the model that was saved during the epoch with the best f1-score of the valodation set. The history is a object that contains the values of all metrics for all epochs. It is used to plot the metrics.

In [ ]:
help(gmlu.train_model)

In [ ]:
save_path = r'fashion_mnist/models'
model, history = gmlu.train_model(model=model, train_ds=train_ds, val_ds=val_ds, save_path=save_path, folder='demonstration', epochs=10, patience=10)

We can use the history object to plot the course of al metrics with the plot_history() function.

In [ ]:
gmlu.plot_history(history)

You can load a trained model directly via the `load_model_from_path`-function. These 3 methods to load a trained model are equivalent. The last line only loads the weights and requires that the fitting model architecture for the weights was build beforehand.

In [ ]:
model = gmlu.load_model_from_path(r'fashion_mnist\models\demonstration\simplenet\best_model')
model = gmlu.load_model_from_path(r'fashion_mnist\models\demonstration\simplenet\best_model.h5')
model.load_weights(r'fashion_mnist\models\demonstration\simplenet\best_model_weights\checkpoint.weights.h5')

### Übungen 3

1. Follow the value of lr (learning rate) over the epochs. What causes it to change?

2. Follow the metrics for the train set and the val set. DId the best epoch for the train set correspond to the best epoch for the val set?

3. Train a simplenet size m multilabel model on the bee_dataset for 10 epochs with img_size 100x50

4. Build a efficientnet size s and a simplenet size l. Train both of them for 2 epochs and compare the time per epoch.

5. Build the dataset with image_size=(128,128,1), train the simplenet size s model with it for 10 epochs and compare the training times and the best metrics.

6. Look into the 'savedir' directory. What is saved there and what does it contain?

## Use the model to predict images


Next we want to use the trained model to predict images. For that we use the make_predictions() function.

We pass the model directly to that function. Or we pass the path to the saved model. 

Then we provide the data we want to predict. We use the test set, but we can also pass a single image or a path to a image fole or a path to a folder with images.

In [ ]:
help(gmlu.make_predictions)

In [ ]:
img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
train_ds, val_ds, test_ds = gmlu.build_img_dataset_pipeline(img_path=img_path, label_path=label_path, img_size=(32, 32, 1), n_classes=10, img_type='png',augment_strength=[1, 0, 0, 0.3, 0.3, 0.3],seed=42,verbose=0)
model = gmlu.load_model_from_path(r'fashion_mnist\models\demonstration\simplenet\best_model')

probabilities, predictions = gmlu.make_predictions(model=model, data=test_ds, img_size=(32,32,1),classification_type='multiclass')
print('probabilities:\n', probabilities)
print('predictions:\n', predictions)

### Übungen 4

1. Predict all the images in the path fashion_mnist/images.

## Evaluate the model

To evaluate the performance of a model we can use the evaluate_model function. This function calculates the metrics of the model on a test dataset. It calculates the metric on the whole dataset and per class and displays them in a bar plot. It also generates the confusion matrix.

In [ ]:
help(gmlu.evaluate_model)

When using the build_img_dataset_pipeline() function to generate the test dataset, it is important to use the same seed as used when training the model. Here we use seed=42 in both locations. This ensures that the data is shuffeled in the same order and the images in the train, val and test sets are the same. 

In [ ]:
img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
train_ds, val_ds, test_ds = gmlu.build_img_dataset_pipeline(img_path=img_path, label_path=label_path, img_size=(32, 32, 1), n_classes=10, img_type='png',seed=42,verbose=0)

In [ ]:
model_path = r'fashion_mnist\models\demonstration\simplenet\best_model'
class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
report_dict = gmlu.evaluate_model(model= model_path, test_ds=test_ds,classification_type='multiclass',class_names=class_names)

### Übungen 5

1. Identify the class that is most often mistaken for a different class. What can you say about the precision and recall values of that class?

2. What does a horizontal class spread in the confusion matrix indicate? A low precision or a low recall?

3. What does a vertical class spread in the confusion matrix indicate? A low precision or a low recall?

4. Why is the accuracy the same for all classes?

5. Evaluate the model for the bee dataset with a confidence threshold of 0.7 and 0.3 and compare which metrics change the most.


## Find best confidence threshold values for model

When we have a multilabel problem we can optimize the individual confidence thresholds per class with the `optimize_conf_thresholds`-function. This optimization has no effect on multiclass problems.

In [ ]:
help(gmlu.optimize_conf_thresholds)

In [ ]:
img_path = r'bee_dataset\images'
label_path = r'bee_dataset\labels.csv'
train_ds,val_ds,_ = gmlu.build_img_dataset_pipeline(img_path=img_path, label_path=label_path, img_size=(100, 50, 3), n_classes=4,val_size=0.3, test_size=0, img_type='jpeg', augment_strength=[1,0,0,0.1,0.1,0.1],seed=42,verbose=1)

For the next cell you need to do Übung 3.3 where you create a model for the multilabel bee-dataset, since the optimization of confidence thresholds is only applicable to multilabel models.

In [ ]:
bee_model_path =r'bee_dataset\models\demonstration\simplenet\best_model'
report = gmlu.evaluate_model(model=bee_model_path, test_ds=val_ds,classification_type='multilabel')
print(report['weighted avg']['f1-score'])

In [ ]:
bee_model_path =r'bee_dataset\models\demonstration\simplenet\best_model'
optimal_thresholds = gmlu.optimize_conf_thresholds(model=bee_model_path, val_ds=val_ds)
print('optimal_thresholds:', optimal_thresholds)
print('len optimal_thresholds:', len(optimal_thresholds))

In [ ]:
report = gmlu.evaluate_model(model=bee_model_path, test_ds=val_ds, conf_thresholds=optimal_thresholds, classification_type='multilabel')
print(report['weighted avg']['f1-score'])

### Übungen 6

1. Optimize the thresholds once for Recall and for Precision and compare the evaluation-results.

## Find the optimal model for a dataset


The performance of a model can vary strongly for different datasets. So if we want the best possible model for specific dataset we want to do a hyperparameter search with find_best_mode().

This compares all the different model architectures with many different configurations to find the best model.

First we build our datasets again, with img_size=(32,32,1). No model can use images with a smaller size except for simplenet with size 's', which can work with images of a minimal size of 8x8. To include all models in the search we have to use a image size of at least 75x75. 

In [ ]:
help(gmlu.find_best_model)

In [ ]:
img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
train_ds, val_ds, test_ds = gmlu.build_img_dataset_pipeline(img_path=img_path, label_path=label_path, img_size=(32, 32, 1), n_classes=10, img_type='png',augment_strength=[1,0,0,0.1,0.1,0.1],verbose=0)

Here we start the hyperparameter search. This Function will run a long time. We set the maximal training epochs per model to 30 to reduce the training time. Ideally the epochs should be 20-60 (dependent on the complexity of the images and the number of classes). The `iterations` parameter sets the number of different hyperparameter-configurations that are tried. This should be minimal 20 but ideally more. This is because the used optimization algorithm optimizes the choice of hyperparameters and needs at least 20 iterations to work as intended (the optimization algorithm is a Tree-structured Parzen Estimator in combination with a Hyperband pruner). If we use less iterations than 20 then we have effectiveley a random search algorithm.

The training progress of the individual iterations is logged in a online dashboard from the `Weights & Biases` library. When you click on the link at "View run at :" you can see live charts of the different training metrics. When you navigate to the root project of the run or click on the link at "View project at" you see the charts of the different metrics of all trials together. This is usefull to quickly see which iteration was the best or to get a sense of which model architecture performs the best for that dataset across different hyperparameter configurations.

You dont need a account to see the dashboard and all logged data gets deleted after 7 days (but you can claim the data when you make a account, then it is permanent).

In [ ]:
save_path = r'fashion_mnist/models/hyperparameter_search'
gmlu.find_best_model(train_ds=train_ds, val_ds=val_ds, test_ds=test_ds, save_path=save_path, max_epochs=10, iterations=20, classification_type='multiclass', models='all', model_sizes=['s','m'],eval_metric='f1_score')

### Übungen 7

1. Go to the wandb dashboard and group the runs once by model_size and by model. What can you generally say about the performance?

2. Go to the wandb dashboard and list the 5 most important parameters and if they contribue positively or negatively.

3. Select one run and go to the logs. Can you find the save path of the trained model and the Host-PC?

4. Start a new hyperparameter search with all models and a specify a project name.

## Validate the performance of a model


To validate the performance of a model we use a technique called cross-validation. We need to do this because the evaluation and training metrics of a model (like val_f1_score) dont tell the whole story, because that is only the performance on a small part of the dataset. Per chance the test or validation sets could only consist of the easiest to classify images, which would lead to the model underperforming in the production environment. 

Cross-validation gives a much better performance indication of the model by splitting the dataset into multiple parts where each part is used once as the test set and the other parts as the train set. By this the model gets tested on the whole dataset not only on a part of it.

In [ ]:
help(gmlu.cross_validation)

Here we will evaluate the first model, that we trained before. It is the MobileNet with the standart configuration. That the model was trained is irrelevant since we retrain the model in the cross-validation. We train the model here only for 10 epochs in 5 folds to make it faster. Standard parameters would be ca. 60 epochs and 5 folds.

Normally one would use the cross-validation to evaluate the performance of a Hyperparameter configuration that was derived by a hyperparameter search.

In [ ]:
img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
model_path= r'fashion_mnist\models\demonstration\simplenet\best_model'
save_path = r'fashion_mnist\cross_validation'

gmlu.cross_validation(model=model_path, img_path=img_path, label_path=label_path, save_path=save_path, folds = 2, epochs = 1, n_classes=10, classification_type='multiclass', img_type='png', augment_strength=0.1)

### Übungen 8

1. Do a cross validation and save the filenames of the folds. Name the first 10 filenames that are used in the first and second fold.

## Export the model to ONNX format

In [ ]:
help(gmlu.export_onnx_model)

We can export a model to the ONNX-format from a path of a saved model. 

In [ ]:
model_path= r'fashion_mnist\models\demonstration\simplenet\best_model'
save_path =  r'fashion_mnist\models\demonstration\ONNX'
gmlu.export_onnx_model(model_path, save_path)

Or we can directly pass a build model. 
When no save-path is provided then the onnx-model will be saved in a standard location. 

In [ ]:
model = gmlu.load_model_from_path(model_path)
gmlu.export_onnx_model(model)

In [ ]:
import onnx
import onnxruntime as ort
import numpy as np

img_path = r'fashion_mnist/images'
label_path = r'fashion_mnist/labels.csv'
train_ds, val_ds, test_ds = gmlu.build_img_dataset_pipeline(img_path=img_path, label_path=label_path, img_size=(32, 32, 1), n_classes=10, img_type='png',augment_strength=[1,0,0,0.1,0.1,0.1],verbose=0)

model = onnx.load(r'fashion_mnist\models\demonstration\ONNX\simplenet\model.onnx')
# Create an ONNX Runtime session
session = ort.InferenceSession(model.SerializeToString())
# Get the input name(s) of the model
input_name = session.get_inputs()[0].name
# Get the output name(s) of the model
output_name = session.get_outputs()[0].name

tf_model = gmlu.load_model_from_path(r'fashion_mnist\models\demonstration\simplenet\best_model')
for batch in test_ds:
    images, labels = batch
    # Get the data as a numpy array
    images = images.numpy()
    images = images.astype(np.float32)
    # Run the prediction
    prediction = session.run([output_name], {input_name: images})[0]
    print('predictions of first batch:\n',prediction[0])
    tf_prediction = tf_model.predict(images)
    print('compare with predictions of tensorflow model:\n',tf_prediction[0])
    break

## Multimodal combination

If we have two modalities of the same objects we can combine the predictions of the modalities to get a better result than using only one. 

For multimodal prediction there are two possibilities: batch prediction and single image prediction

When using batch prediction we are only interested what classes are generally present in the batches and not at what position in the batch they are positioned. This is usefull when there are many images of the same object and we want to know if there is a defect present anywhere. Or generally when the images can be grouped to larger categories (like Laufmeter in DIS project 149). For that we use the gmlu.multimodal_prediction_batch() function.

When there is a perfect parity per image so that we want to combine only the predictions of the two images of the modalities we can use the gmlu.multimodal_prediction_single() function. This requires that there is the same number of images in the batches of the modalities. 

### Using multimodal combination on batches of images

In [ ]:
help(gmlu.multimodal_prediction_batch)

We first load the two models that should be used for predicting.

In [ ]:
x_model_path = r'models/x_model'
o_model_path = r'models/o_model'
x_model = gmlu.load_model_from_path(x_model_path)
o_model = gmlu.load_model_from_path(o_model_path)

Then we construct the data for the multimodal prediction. We load three batches of images and labels. 

In [ ]:
batches_path = 'Rolle_01935'
x_batches = []
o_batches = []
labels=[]
folders = os.listdir(batches_path)
for folder in folders:
    x_files = os.listdir(os.path.join(batches_path, folder, 'XDIS'))
    o_files = os.listdir(os.path.join(batches_path, folder, 'ODIS'))
    x_files = [os.path.join(batches_path, folder, 'XDIS', file) for file in x_files]
    o_files = [os.path.join(batches_path, folder, 'ODIS', file) for file in o_files]
    x_batch = [np.array(Image.open(file).convert('L').resize((128,128), Image.LANCZOS)) for file in x_files]
    x_batch = np.expand_dims(x_batch, axis=-1)
    print('x_batch size: ',np.shape(x_batch))
    x_batches.append(x_batch)
    
    o_batch = [np.array(Image.open(file).convert('L').resize((128,128), Image.LANCZOS)) for file in o_files]
    o_batch = np.array(o_batch)
    o_batch = np.expand_dims(o_batch, axis=-1)
    print('o_batch size: ',np.shape(o_batch))   
    o_batches.append(o_batch)   
    
    label_file = os.path.join(batches_path, folder, 'labels', 'classes.csv')
    label = pd.read_csv(label_file, header=None)
    labels.append(label.values[0].tolist())
    print('label: ',label.values.tolist())
print('nr of x_batches: ',len(x_batches))
print('nr of o_batches: ',len(o_batches))
print(labels)

The function gmlu.multimodal_prediction_batch only takes in a single batch of data per modality and produces a single prediction and a single probability vector. 

In [ ]:
preds, probs,_,_ = gmlu.multimodal_prediction_batch(x_model, o_model, x_batches[0], o_batches[0], n_classes=10,classification_type='multilabel')
print('preds: ',preds)
print('probs: ',probs)

When we want to predict many batches we predict them in a loop and save the predictions in a list.

To evaluate the prediction we can use the gmlu.evaluate_predictions() function.

In [ ]:
help(gmlu.evaluate_predictions)

In [ ]:
predictions = []
for i in range(len(x_batches)):
    preds, probs,_,_ =gmlu.multimodal_prediction_batch(x_model, o_model, x_batches[i], o_batches[i], n_classes=10,classification_type='multilabel')
    predictions.append(preds)
    
print('preds: ',predictions)
predictions = gmlu.one_hot_encode(predictions, 10, classification_type='multilabel')
print('predictions: ',predictions)
print('labels: ',labels)
report = gmlu.evaluate_predictions(predictions, labels, classification_type='multilabel')

In [ ]:
optimal_Ts, optimal_thresholds = gmlu.optimize_multimodal_parameters(x_model, o_model, x_batches, o_batches, labels, n_classes=10, classification_type='multilabel',type='batch')

In [ ]:
print('optimal_Ts: ',optimal_Ts)
print('optimal_thresholds: ',optimal_thresholds)

predictions = []
for i in range(len(x_batches)):
    preds, probs,_,_ =gmlu.multimodal_prediction_batch(x_model, o_model, x_batches[i], o_batches[i], n_classes=10,classification_type='multilabel',conf_thresholds=optimal_thresholds, T=optimal_Ts)
    predictions.append(preds)
    
print('preds: ',predictions)
predictions = gmlu.one_hot_encode(predictions, 10, classification_type='multilabel')
print('predictions: ',predictions)
print('labels: ',labels)
report = gmlu.evaluate_predictions(predictions, labels, classification_type='multilabel')

### Using multimodal combination of individual images

In [ ]:
train, val = tf.keras.datasets.cifar10.load_data()
#split datasets into images and labels
train_images, train_labels = train
#split images into channels
train_images1 = train_images[:,:,:,0:1]
train_images2 = train_images[:,:,:,1:2]
train_labels = gmlu.one_hot_encode(train_labels, 10)
val_images, val_labels = val
val_images1 = val_images[:,:,:,0:1]
val_images2 = val_images[:,:,:,1:2]
val_labels = gmlu.one_hot_encode(val_labels, 10)
print('shape images1: ',np.shape(train_images1))
print('shape images2: ',np.shape(train_images2))
print('shape labels: ',np.shape(train_labels))
print(train_labels[:10])

In [ ]:
save_path = r'cifar10/models'
model1 = gmlu.build_model(input_shape=(32, 32, 1), n_classes=10, model_type='simplenet',model_size='s', classification_type='multiclass',classifier_neurons=[256,64])
model1, history1 = gmlu.train_model(model=model1, train_ds=(train_images1,train_labels), val_ds=(val_images1,val_labels), save_path=save_path, folder='channel1', epochs=10, patience=10)

model2 = gmlu.build_model(input_shape=(32, 32, 1), n_classes=10, model_type='simplenet',model_size='s', classification_type='multiclass',classifier_neurons=[256,64])
model2, history2 = gmlu.train_model(model=model2,  train_ds=(train_images2,train_labels), val_ds=(val_images2,val_labels), save_path=save_path, folder='channel2', epochs=10, patience=10)

In [ ]:
model1 = gmlu.load_model_from_path(r'cifar10\models\channel1\simplenet\best_model')
model2 = gmlu.load_model_from_path(r'cifar10\models\channel2\simplenet\best_model')
preds, probs,_,_ =gmlu.multimodal_prediction_single(model1, model2, val_images1, val_images2, n_classes=10, classification_type='multiclass',T=1000)
predictions = gmlu.one_hot_encode(preds, 10, classification_type='multiclass')
report = gmlu.evaluate_predictions(predictions, val_labels, classification_type='multiclass')

In [ ]:
optimal_Ts, optimal_thresholds = gmlu.optimize_multimodal_parameters(model1, model2, val_images1, val_images2, val_labels, n_classes=10, classification_type='multiclass',type='single')
preds, probs,_,_ =gmlu.multimodal_prediction_single(model1, model2, val_images1, val_images2, n_classes=10, classification_type='multiclass',T=optimals_Ts)
predictions = gmlu.one_hot_encode(preds, 10, classification_type='multiclass')
report = gmlu.evaluate_predictions(predictions, val_labels, classification_type='multiclass')

In [ ]:
probs1, preds1 = gmlu.make_predictions(model1, val_images1, classification_type='multiclass')
print('preds1: ',preds1[:10])
predictions1 = gmlu.one_hot_encode(preds1, 10, classification_type='multiclass')
report1 = gmlu.evaluate_predictions(predictions1, val_labels, classification_type='multiclass')

In [ ]:
probs2, preds2 = gmlu.make_predictions(model2, val_images2, classification_type='multiclass')
print('preds2: ',preds2[:10])
predictions2 = gmlu.one_hot_encode(preds2, 10, classification_type='multiclass')
report2 = gmlu.evaluate_predictions(predictions2, val_labels, classification_type='multiclass')